<h1><center> Santa-2020 - who's lucky? (leaderboard analysis) </center></h1>

<h2><center> <img src="https://upload.wikimedia.org/wikipedia/commons/6/68/Xmasart_%2857%29.jpg" alt="Christmas img"></center></h2>

It's quite obvious that luck plays an important role in this competition. This notebook shows that most medal-winning teams have small number of "medal zone" agents. That is, these are lucky outliers (repeated submissions didn't achieve the same score). However, this effect is much smaller than in [Rock, Paper, Scissors](https://www.kaggle.com/demche/rock-paper-scissors-leaderboard-eda) competition.

[@dmitriyguller](https://www.kaggle.com/dmitriyguller) quite precisely formulated this issue: 
> The nature of the leaderboard also exaggerates luck, because it rewards the extreme deviation from expected performance, not the expected performance itself. An agent that scores 1100 99% of the time, but 1350 the other 1% of the time, is preferable to the agent who gets 1325 every single time.

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
from kaggle_environments import list_episodes
from IPython.display import display, Markdown
pd.set_option("display.max_rows", 200)
pd.options.display.float_format = '{:,.2f}'.format
warnings.filterwarnings('ignore')

In [ ]:
!wget "https://www.kaggle.com/c/santa-2020/leaderboard.json?includeBeforeUser=true&includeAfterUser=false" -O leaderboard.json

In [ ]:
with open("leaderboard.json") as f:
    jsn = json.load(f)
leaderboard = pd.DataFrame(columns = ["team_name", "team_id", "score", "n_agents", "team_rank"])
for user in jsn["beforeUser"]+jsn["afterUser"]:
    leaderboard = leaderboard.append({"team_name": user["teamName"], 
                                      "team_id": user["teamId"], 
                                      "score": user["score"], 
                                      "n_agents": user["entries"],
                                     "team_rank": user["rank"]}, 
                                     ignore_index=True)
leaderboard[["score", "n_agents", "team_rank"]] = leaderboard[["score", "n_agents", "team_rank"]].apply(pd.to_numeric)
gold_min_score = leaderboard.sort_values("score", ascending=False)["score"][10]
silver_min_score = leaderboard.sort_values("score", ascending=False)["score"][49]
bronze_min_score = leaderboard.sort_values("score", ascending=False)["score"][99]

In [ ]:
episodes = pd.read_csv("../input/meta-kaggle/Episodes.csv")
gaps = sorted(set(range(episodes[episodes["CompetitionId"] == 24539]["Id"].min(), episodes["Id"].max() + 1)) - set(episodes["Id"].values), reverse=True)
episodes = episodes.loc[episodes["CompetitionId"] == 24539]
episodes["CreateTime"] = pd.to_datetime(episodes["CreateTime"], format="%m/%d/%Y %H:%M:%S")
episodes = episodes[["Id", "CreateTime"]]

episode_agents = pd.read_csv("../input/meta-kaggle/EpisodeAgents.csv")
episode_agents = pd.merge(episode_agents, episodes, left_on="EpisodeId", right_on="Id")
episode_agents = episode_agents[["EpisodeId", "CreateTime", "SubmissionId", "UpdatedScore"]]
episode_agents = episode_agents.drop_duplicates()
episode_agents["date"] = episode_agents["CreateTime"].dt.date
agents_mapping = pd.DataFrame(columns = ["team_id", "submission_id", "submission_dt"])

episodes_to_consider = episode_agents[episode_agents["EpisodeId"].isin(episodes["Id"])].groupby(["SubmissionId"])["EpisodeId"].max().to_list()
for i in range(0, len(episodes_to_consider), 1000):
    batch = episodes_to_consider[i:i + 1000]
    try:
        resp = list_episodes(batch)  
        for episode in resp["result"]["submissions"]:
            agents_mapping = agents_mapping.append({"team_id": episode["teamId"],
                                "submission_id":  episode["id"] ,
                                "submission_dt": datetime.datetime.strptime(episode["dateSubmitted"][:19], "%Y-%m-%dT%H:%M:%S")
                               }, ignore_index=True)
        del episode, batch
    except Exception as ex:
        print("Error:", ex)
        continue

for i in range(0, len(gaps), 1000):
    batch = gaps[i:i + 1000]
    try:
        resp = list_episodes(batch)      
        if len(resp["result"]["episodes"]) != 0:
            for episode in resp["result"]["episodes"]:
                if episode["competitionId"] == 24539:
                    EpisodeId = episode["id"]
                    for agent in episode["agents"]:
                        submissionId = agent["submissionId"]
                        updatedScore = agent["updatedScore"]
                        CreateTime = datetime.strptime(episode["createTime"][:19], "%Y-%m-%dT%H:%M:%S")
                        episode_agents = episode_agents.append({"EpisodeId": EpisodeId,
                                                    "CreateTime": CreateTime,
                                                    "SubmissionId": submissionId,
                                                    "UpdatedScore": updatedScore
                                                    }, ignore_index=True)           
            for episode in episodes["result"]["submissions"]:
                agents_mapping = agents_mapping.append({"team_id": episode["teamId"],
                                    "submission_id":  episode["id"] ,
                                    "submission_dt": datetime.datetime.strptime(episode["dateSubmitted"][:19], "%Y-%m-%dT%H:%M:%S")
                                   }, ignore_index=True)
            del episode, batch
    except Exception as ex:
        print("Error:", ex)
        continue
        
agents_mapping = agents_mapping.drop_duplicates(subset=["submission_id"])
episode_agents = episode_agents[episode_agents["SubmissionId"].isin(agents_mapping["submission_id"])]
episode_agents = episode_agents.drop_duplicates()
agents = episode_agents.loc[episode_agents.groupby("SubmissionId").CreateTime.idxmax()].dropna(subset=["UpdatedScore"]).\
    loc[:, ["SubmissionId", "UpdatedScore"]].reset_index(drop=True)
agents.columns = ["submission_id", "score"]
agents = pd.merge(agents, agents_mapping, on="submission_id", how="left")
agents = agents.drop_duplicates(subset=["submission_id"])
agents = pd.merge(agents, leaderboard.loc[:, ["team_name", "team_id"]], on="team_id", how="left")
agents["medal"] = ["gold" if x >= gold_min_score else "silver" if x >= silver_min_score else "bronze" if x >= bronze_min_score else "no medal" \
     for x in agents["score"]]

# 1. Score distribution

In [ ]:
plt.figure(figsize=(25,8))
plt.hist(leaderboard["score"], color="lightsteelblue", bins=200)
plt.axvline(x=gold_min_score, color="gold")
plt.axvline(x=silver_min_score, color="silver")
plt.axvline(x=bronze_min_score, color="peru")
plt.xlabel("Team score")
plt.ylabel("Number of teams")
plt.legend(title="Team score distribution (vertical lines are medal thresholds)", loc="upper center", title_fontsize=25)
plt.show()

In [ ]:
plt.figure(figsize=(25,8))
plt.hist(leaderboard["score"][leaderboard["score"] > 1000], color="thistle", bins=120)
plt.axvline(x=gold_min_score, color="gold")
plt.axvline(x=silver_min_score, color="silver")
plt.axvline(x=bronze_min_score, color="peru")
plt.xlabel("Team score")
plt.ylabel("Number of teams")
plt.legend(title="Team score distribution (teams with score > 1000, vertical lines are medal thresholds)", loc="upper center", title_fontsize=25)
plt.show()

In [ ]:
leaderboard["score"].describe()

In [ ]:
stat_santa = episode_agents.groupby(["date"]).agg({"UpdatedScore": [np.max, np.min, np.mean, np.median]}, axis="columns")
stat_santa.columns = stat_santa.columns.droplevel(0)
stat_santa.plot(figsize=(25,10), title="Score summary statistics for individual agents", colormap="Spectral")
plt.show()

# 2. Number of submissions for medal-winning teams

In [ ]:
plt.figure(figsize=(25,8))
plt.hist([leaderboard.sort_values("score", ascending=False)["n_agents"][:10],
          leaderboard.sort_values("score", ascending=False)["n_agents"][11:51],
          leaderboard.sort_values("score", ascending=False)["n_agents"][51:101]],
         label=["gold-winning team", "silver-winning team", "bronze-winning team"],
         color= ["gold", "silver", "peru"], bins=50, stacked=True, alpha=0.7)
plt.xlabel("Number of submissions")
plt.ylabel("Number of teams")
plt.legend(title="Total number of submissions for medal-winning teams", loc="upper center", title_fontsize=20)
plt.show()

# 3. Medal zone agents

In [ ]:
agents_bronze = agents[agents["medal"] == "bronze"].sort_values(by=["score"], ascending=False).reset_index()["team_name"].\
value_counts().reset_index().rename(columns={"index": "team", "team_name": "bronze"})
agents_silver = agents[agents["medal"] == "silver"].sort_values(by=["score"], ascending=False).reset_index()["team_name"].\
value_counts().reset_index().rename(columns={"index": "team", "team_name": "silver"})
agents_gold = agents[agents["medal"] == "gold"].sort_values(by=["score"], ascending=False).reset_index()["team_name"].\
value_counts().reset_index().rename(columns={"index": "team", "team_name": "gold"})
medal_zone_agents = pd.merge(pd.merge(agents_gold, agents_silver, on="team", how="outer"), agents_bronze, on="team", how="outer").fillna(0)
medal_zone_agents["bronze"] = medal_zone_agents["bronze"].astype(int)
medal_zone_agents["silver"] = medal_zone_agents["silver"].astype(int)
medal_zone_agents["gold"] = medal_zone_agents["gold"].astype(int)
medal_zone_agents.head(100).style.background_gradient(cmap="YlGn")

# 4. Time of submission by medal zone

[@pedram](https://www.kaggle.com/pedram) [suggested](https://www.kaggle.com/c/santa-2020/discussion/214271) that newly written agents can possibly get into the top. Right now there is no apparent relationship between submission time and score. 

In [ ]:
agents["medal_color"] = ["peru" if x == "bronze" else "lightblue" if x == "no medal" else x for x in agents["medal"]]
for medal_type, group in agents.groupby("medal"):
    display(Markdown("Medal: " + str(medal_type)))
    plt.figure(figsize=(20, 10))
    plt.scatter(group["submission_dt"], group["score"], c=group["medal_color"])
    plt.title("Medal: " + str(medal_type))
    plt.show()

# 5. Top-10 agents for medal-winning teams

In [ ]:
top_teams_agents = agents[agents["team_id"].isin(leaderboard.sort_values(by=["score"], ascending=False).head(100)["team_id"])].reset_index()
top_teams_agents = pd.merge(top_teams_agents, leaderboard[["team_id", "team_rank"]], on=["team_id"], how="left")

In [ ]:
top_teams_agents_best10 = top_teams_agents.sort_values("score",ascending = False).groupby("team_id").head(10).reset_index(drop=True)
top_teams_agents_best10 = top_teams_agents_best10.loc[:, ["team_name", "score"]]
top_teams_agents_best10["rank"] = top_teams_agents_best10.groupby("team_name")["score"].rank("dense", ascending=False).astype(int)
top_teams_agents_best10.pivot(index="team_name", columns="rank", values="score").sort_values(1,ascending = False).style.background_gradient(cmap="YlOrBr")

# 6. Best agent vs top agents

Best agent is clearly outlier if there is significant difference between its score and top agents' score.

In [ ]:
best1_agents = agents.sort_values("score",ascending = False).groupby("team_name").head(1).reset_index(drop=True).\
    groupby("team_name").agg({"score": np.mean}).rename(columns={"score": "best agent"})
best10_agents = agents.sort_values("score",ascending = False).groupby("team_name").head(10).reset_index(drop=True).\
    groupby("team_name").agg({"score": np.mean}).rename(columns={"score": "top 10 agents"})
best30_agents = agents.sort_values("score",ascending = False).groupby("team_name").head(30).reset_index(drop=True).\
    groupby("team_name").agg({"score": np.mean}).rename(columns={"score": "top 30 agents"})
best_agents = pd.merge(pd.merge(pd.merge(best1_agents, best10_agents, on=["team_name"]), best30_agents, on=["team_name"]), 
                leaderboard.loc[:, ["team_name", "n_agents"]], on=["team_name"])
best_agents["difference best - top 10"] = best_agents.apply(lambda x: x["best agent"] - x["top 10 agents"], axis=1)
best_agents["difference best - top 30"] = best_agents.apply(lambda x: x["best agent"] - x["top 30 agents"], axis=1)
best_agents["medal"] = ["gold" if x >= gold_min_score else "silver" if x >= silver_min_score else "peru" if x >= bronze_min_score else "lightblue" \
     for x in best_agents["best agent"]]

In [ ]:
plt.figure(figsize=(25,15))
plt.scatter(best_agents["best agent"], best_agents["top 10 agents"], c=best_agents["medal"], alpha=0.7)
plt.xlabel("Best agent score")
plt.ylabel("Top-10 agents score (mean)")
plt.legend(title="Top-10 agents score (mean) vs best agent score", loc="lower center", title_fontsize=25)
plt.show()

In [ ]:
plt.figure(figsize=(25,8))
plt.hist(best_agents["difference best - top 10"][best_agents["difference best - top 10"] != 0], color="lightcoral", bins=200)
plt.xlabel("Difference between best agent and top 10 agents")
plt.ylabel("Number of teams")
plt.legend(title="Difference between top-10 agents score (mean) and best agent score", loc="upper center", title_fontsize=25)
plt.show()

In [ ]:
plt.figure(figsize=(25,15))
plt.scatter(best_agents["best agent"], best_agents["top 30 agents"], c=best_agents["medal"], alpha=0.7)
plt.xlabel("Best agent score")
plt.ylabel("Top-30 agents score (mean)")
plt.legend(title="Top-30 agents score (mean) vs best agent score", loc="lower center", title_fontsize=25)
plt.show()

In [ ]:
plt.figure(figsize=(25,8))
plt.hist(best_agents["difference best - top 30"][best_agents["difference best - top 30"] != 0], color="orange", bins=200)
plt.xlabel("Difference between best agent and top 30 agents")
plt.ylabel("Number of teams")
plt.legend(title="Difference between top-30 agents score (mean) and best agent score", loc="upper center", title_fontsize=25)
plt.show()

Difference for medal-winning teams:

In [ ]:
best_agents[best_agents["team_name"].isin(leaderboard.sort_values("score", ascending=False)["team_name"][0:99])].\
    loc[:, ["team_name", "best agent", "top 10 agents", "top 30 agents", "difference best - top 10", 
             "difference best - top 30", "n_agents"]].sort_values("best agent", ascending=False).reset_index(drop=True)